In [2]:
class VariableElimination:
    @staticmethod
    def inference(factor_list, query_variables,
                  ordered_list_of_hidden_variables, evidence_list):
        for ev in evidence_list:
            # Your code here
            tmp_factor_list = factor_list.copy()
            for factor in factor_list:
                if ev in factor.var_list:
                    tmp_factor_list.remove(factor)
                    tmp_factor_list.append(factor.restrict(ev, evidence_list[ev]))
            factor_list = tmp_factor_list
            # my code end
        for var in ordered_list_of_hidden_variables:
            # Your code here
            new_factor = None
            tmp_factor_list = factor_list.copy()
            for factor in factor_list:
                if var in factor.var_list:
                    tmp_factor_list.remove(factor)
                    new_factor = factor if new_factor == None else new_factor.multiply(factor)
            tmp_factor_list.append(new_factor.sum_out(var))
            factor_list = tmp_factor_list
            # My code end
        print("RESULT: ")
        res = factor_list[0];
        for factor in factor_list[1:]:
            res = res.multiply(factor)
        total = sum(res.cpt.values())
        res.cpt = {k: v / total for k, v in res.cpt.items()}
        res.print_inf()
        return res

    @staticmethod
    def print_factors(factor_list):
        for factor in factor_list:
            factor.print_inf()

class Util:
    @staticmethod
    def to_binary(num, len):
        return format(num, '0' + str(len) + 'b')


def match(key1, key2, var_share_idx_pair):
    tmp, s = '',0
    for i, pair in enumerate(var_share_idx_pair):
        if key1[pair[0]] != key2[pair[1]]: return False, None
        tmp += key2[s:pair[1]]
        s = pair[1] + 1
    return True, key1 + tmp + key2[s:]

class Node:
    def __init__(self, name, var_list):
        self.name = name
        self.var_list = var_list
        self.cpt = {}

    def set_cpt(self, cpt):
        self.cpt = cpt

    def print_inf(self):
        print("Name = " + self.name)
        print(" vars " + str(self.var_list))
        for key in self.cpt:
            print("   key: " + key + " val : " + str(self.cpt[key]))
        print()
    
    def multiply(self, factor):
        '''function that multiplies with another factor'''
        # Your code here
        var_share = [var for var in factor.var_list if (var in self.var_list)]
        new_list = self.var_list + [var for var in factor.var_list if var not in var_share]

        var_share_idx_pair = [[self.var_list.index(var), factor.var_list.index(var)] for var in var_share]
        new_cpt = {}
        for key1, pro1 in self.cpt.items():
            for key2, pro2 in factor.cpt.items():
                isMatch, new_key = match(key1, key2, var_share_idx_pair)
                if isMatch: new_cpt[new_key] = pro1 * pro2
        #My code end
        new_node = Node('f' + str(new_list), new_list)
        new_node.set_cpt(new_cpt)
        return new_node

    def sum_out(self, variable):
        '''function that sums out a variable given a factor'''
        # Your code here
        idx = self.var_list.index(variable)
        new_var_list = self.var_list.copy()
        new_var_list.remove(variable)
        new_cpt = {}
        for key, pro in self.cpt.items():
            new_cpt[key[:idx] + key[idx+1:]] = new_cpt.get(key[:idx] + key[idx+1:],0) + pro
        # My code end
        new_node = Node('f' + str(new_var_list), new_var_list)
        new_node.set_cpt(new_cpt)
        return new_node

    def restrict(self, variable, value):
        '''function that restricts a variable to some value
        in a given factor'''
        # Your code here
        idx = self.var_list.index(variable)
        new_var_list = self.var_list.copy()
        new_var_list.remove(variable)
        new_cpt = {}
        for key, pro in self.cpt.items():
            if key[idx] == chr(value + ord('0')):
                new_cpt[key[:idx] + key[idx+1:]] = pro
        ### My ocde end
        new_node = Node('f' + str(new_var_list), new_var_list)
        new_node.set_cpt(new_cpt)
        return new_node


# Create nodes for Bayes Net
B = Node('B', ['B'])
E = Node('E', ['E'])
A = Node('A', ['A', 'B', 'E'])
J = Node('J', ['J', 'A'])
M = Node('M', ['M', 'A'])

# Generate cpt for each node
B.set_cpt({'0': 0.999, '1': 0.001})
E.set_cpt({'0': 0.998, '1': 0.002})
A.set_cpt({'111': 0.95, '011': 0.05, '110': 0.94, '010': 0.06,
           '101':0.29, '001': 0.71, '100': 0.001, '000': 0.999})
J.set_cpt({'11': 0.9, '01': 0.1, '10': 0.05, '00': 0.95})
M.set_cpt({'11': 0.7, '01': 0.3, '10': 0.01, '00': 0.99})

print("P(A) **********************")
p1 = VariableElimination.inference([B, E, A, J, M], ['A'], ['B', 'E', 'J', 'M'], {}).cpt['1']

print("P(J, ~M) **********************")
p2 = VariableElimination.inference([B, E, A, J, M], ['J', 'M'], ['E', 'A', 'B'], {}).cpt['10']

print("P(A | J, ~M) **********************")
p3 = VariableElimination.inference([B, E, A, J, M], ['A'], ['E', 'B'], {'J':1, 'M':0}).cpt['1']

print("P(B | A) **********************")
p4 = VariableElimination.inference([B, E, A, J, M], ['B'], ['E','J', 'M'], {'A':1}).cpt['1']

print("P(B | J, ~M) **********************")
p5 = VariableElimination.inference([B, E, A, J, M], ['B'], ['E', 'A'], {'J':1, 'M':0}).cpt['1']

print("P(J, ~M | ~B) **********************")
p6 = VariableElimination.inference([B, E, A, J, M], ['J', 'M'], ['E', 'A'], {'B':0}).cpt['10']

P(A) **********************
RESULT: 
Name = f['A']
 vars ['A']
   key: 1 val : 0.0025164420000000002
   key: 0 val : 0.997483558

P(J, ~M) **********************
RESULT: 
Name = f['J', 'M']
 vars ['J', 'M']
   key: 11 val : 0.002084100239
   key: 10 val : 0.05005487546100001
   key: 01 val : 0.009652244741000002
   key: 00 val : 0.9382087795590001

P(A | J, ~M) **********************
RESULT: 
Name = f['A']
 vars ['A']
   key: 1 val : 0.013573889331307631
   key: 0 val : 0.9864261106686925

P(B | A) **********************
RESULT: 
Name = f['B']
 vars ['B']
   key: 0 val : 0.626448771718164
   key: 1 val : 0.373551228281836

P(B | J, ~M) **********************
RESULT: 
Name = f['B']
 vars ['B']
   key: 0 val : 0.9948701418665987
   key: 1 val : 0.0051298581334013015

P(J, ~M | ~B) **********************
RESULT: 
Name = f['J', 'M']
 vars ['J', 'M']
   key: 11 val : 0.001493351
   key: 10 val : 0.049847948999999996
   key: 01 val : 0.009595469
   key: 00 val : 0.939063231



In [3]:
print("P(A) =", p1)

print("P(J, ~M) =", p2)

print("P(A | J, ~M) =", p3)

print("P(B | A) =", p4)

print("P(B | J, ~M) =", p5)

print("P(J, ~M | ~B) =", p6)

P(A) = 0.0025164420000000002
P(J, ~M) = 0.05005487546100001
P(A | J, ~M) = 0.013573889331307631
P(B | A) = 0.373551228281836
P(B | J, ~M) = 0.0051298581334013015
P(J, ~M | ~B) = 0.049847948999999996
